In [4]:
import os
import numpy as np

import matplotlib.pyplot as plt
from ipywidgets import interact, widgets
import mpld3

dataset = "Peat6"

result_path = "/home/luke/data/Model/results/"
data_path = "/home/luke/data/MATRIX_data/"

path = result_path + dataset
path2 = data_path + dataset

In [7]:
x_sol = np.load(path + '/sol.npy')
full_ref = np.load(path + '/full_ref.npy')
full_obs = np.load(path + '/full_resid_spectra.npy')
obs_spec = np.load(path + '/obs.npy')
W = np.load(path + '/W.npy')
W_full = np.load(path + '/W_full.npy')

In [9]:
def PlotSpectralResiduals(full_ref_spec, full_obs_spec, W_full, x_sol, sigma, Compounds, dataset: str, t = None):

    y_model, y, y_model_err = inversion_residual(full_ref_spec, full_obs_spec, x_sol, np.sqrt(sigma))

    Nl = full_ref_spec.shape[1]
    Nt = full_obs_spec.shape[0]

    for j, key in enumerate(Compounds):

        species_arr = x_sol[j * Nt:(j + 1) * Nt]
        t = np.argmax(species_arr)
        y_model_sel, y_sel, y_model_err_sel = y_model[t*Nl:(t+1)*Nl], y[t*Nl:(t+1)*Nl], y_model_err[t*Nl:(t+1)*Nl]

        num = len(Compounds[key]['bounds'])
        fig, axs = plt.subplots(num, 2, figsize=(12, 4*num), gridspec_kw={'width_ratios': [3, 1]})
        if num == 1:
            axs = [axs]

        fig.suptitle(key, x=0.05, y=0.95, ha='left', fontsize=16)  # Add figure text at the top left
        fig.text(0.02, 0.5, 'Absorbance', va='center', ha='left', rotation='vertical', fontsize=14)
        fig.text(0.695, 0.5, 'Observed - Modelled Spectra', va='center', ha='left', rotation='vertical', fontsize=14)

        for i, bound in enumerate(Compounds[key]['bounds']):
            
            max_points = []
            min_points = []

            for b in Compounds[key]['bounds']:
                max_points.append(np.max(y_model_sel[np.where((W_full >= b[0]) & (W_full <= b[1]))]))
                min_points.append(np.min(y_model_sel[np.where((W_full >= b[0]) & (W_full <= b[1]))]))

            # Plot on the left side
            axs[i][0].plot(W_full, y_sel, color='red', linewidth=4, label='Observed Spectra')
            axs[i][0].plot(W_full, y_model_sel, '-.', color='k', label='Modelled Spectra')
            axs[i][0].fill_between(W_full, y_model_sel - y_model_err_sel, y_model_sel + y_model_err_sel, color='gray', alpha=0.5)
            axs[i][0].set_xlim(bound[0], bound[1])
            axs[i][0].set_ylim(np.min(min_points)-0.01, np.max(max_points)+0.01)
            axs[i][0].tick_params(axis='both', labelsize=12)

            # Add custom title on the right side
            axs[i][0].text(0.75, 1.05, str(bound[0])+' - '+str(bound[1])+' cm$^{-1}$', transform=axs[i][0].transAxes, va='center', ha='left', fontsize=14, rotation='horizontal', fontstyle='italic')

            # Plot histogram on the right side
            diff = y_sel[np.where((W_full >= bound[0]) & (W_full <= bound[1]))] - y_model_sel[np.where((W_full >= bound[0]) & (W_full <= bound[1]))]
            axs[i][1].hist(diff, bins=20,  alpha=0.7, color='#3498db', edgecolor='black', orientation='horizontal')
            axs[i][1].axhline(y=0, color='black', linestyle='-', linewidth=1)
            axs[i][1].grid(axis='x', linestyle='--', alpha=0.7)
            axs[i][1].tick_params(axis='both', labelsize=12)

        axs[len(Compounds[key]['bounds'])-1][0].set_xlabel('Wavenumber / cm$^{-1}$', fontsize=14)
        axs[len(Compounds[key]['bounds'])-1][1].set_xlabel('Frequency', fontsize=14)
        axs[0][0].legend(fontsize=14)

        # Adjust layout to prevent clipping of titles
        plt.tight_layout(rect=[0.03, 0, 0.98, 0.95])
        plt.subplots_adjust(wspace=0.25)

        # Show the plot
        plt.savefig('/home/luke/data/Model/plots/'+ dataset + '/Residuals/' + key + '.png')
        plt.show()

    return

In [ ]:
PlotSpectralResiduals(full_ref, obs_spec, W_full, x_sol, sigma, Compounds, dataset: str, t = None)